Step 1: install dependencies

In [15]:
!pip3 install tira snorkel wows-eval textdistance rank-bm25

In [16]:
!pip3 uninstall -y tira wows-eval

Found existing installation: tira 0.0.156
Uninstalling tira-0.0.156:
  Successfully uninstalled tira-0.0.156
Found existing installation: wows-eval 0.0.6
Uninstalling wows-eval-0.0.6:
  Successfully uninstalled wows-eval-0.0.6


In [17]:
!pip3 install wows-eval>=0.0.6

In [18]:
!pip3 install python-terrier==0.10.0

In [19]:
!pip3 uninstall numpy -y
!pip3 install numpy --upgrade --force-reinstall


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tira 0.0.156 requires numpy==1.*, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.


In [ ]:
!pip3 install numpy==1.26.4 --force-reinstall
import os
os.kill(os.getpid(), 9)

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [30]:
import pyterrier as pt
from tqdm import tqdm
if not pt.started():
    pt.init()

from tira.rest_api_client import Client
from wows_eval import evaluate as wows_evaluate
import pandas as pd
from jnius import autoclass
import numpy as np

# For measuring consumed resources (e.g., GPU, CPU, RAM, etc.)
from tirex_tracker import tracking, ExportFormat

pd.set_option('display.max_colwidth', None)

# Dataset IDs visible at https://archive.tira.io/datasets?query=wows-eval
DATASET_ID = 'wows-eval/pointwise-20250309-test'
tira = Client()
input_data = tira.pd.inputs(DATASET_ID)

In [31]:
input_data

,id,query,unknown
0,e1786a4d-2b3d-4fcf-ad84-443acf3696e2,journalist risks,"IPI Death Watch: 45 journalists killed in 2021 - International Press Institute\nNewsroom\nIPI Death Watch: 45 journalists killed in 2021\nViolence against journalists remains global challenge\nIPI Contributor Anne ter Rele\nDec 25, 2016\nSHARE\n In 2021, a total of 45 journalists were killed in connection with their work, IPI research revealed today. The sombre tally reflects the continued risks of doing journalism and reaffirms journalist safety as a global challenge. IPI calls on authorities to end impunity for these crimes and to ensure the protection of journalists, who must be able to do their work freely and safely.\n Today, the IPI global network published its yearly Death Watch. IPI’s research shows that since the beginning of 2021, a total of 45 journalists were killed in connection with their work, or lost their lives on assignment. Of these 45 journalists, 40 were male and five were female. A total of 28 were targeted due to their work, while three were killed while covering conflict, two lost their lives covering civil unrest, and one journalist was killed while on assignment."
1,8e64b35c-60be-4292-9542-c4639a749336,journalist risks,"Two journalists were killed in the MENA region, both in Yemen, while nine journalists were killed in Sub-Saharan Africa, most of whom in the Democratic Republic of Congo (3), followed by Burkina Faso and Somalia (both 2).\n Like last year, more journalists were killed in Mexico (7) in 2021 than any other country in the world. All seven cases were targeted killings. According to IPI’s analysis, journalists researching local politics and organized crime, including drug trafficking, are especially at risk. Most of the targeted journalists were examining these or related topics.\n Another explanation for the high number of killed journalists is the level of impunity. According to ARTICLE 19 Mexico, in only one of these seven cases have suspects have been arrested. The continued high number of killings confirms Mexico’s status as one of the deadliest countries for journalists to work. Despite this tragic status quo, the government there has decided to stop funds allocated for upholding the Law for the Protection of Human Rights Defenders and Journalists (LPPDHP).\n After Mexico, Afghanistan and India are the two next-deadliest countries, with six killings each. In Afghanistan, most killings occurred in relation to the violent conflict as a result of the Taliban’s takeover of the country this summer."
2,a8ff3f18-9b80-40b9-b529-799ed3367980,journalist risks,"Importantly, journalists were also murdered in countries with relatively high levels of press freedom, which arguably shows the global nature of the risks of doing journalism. For example, high-risk crime journalist Peter R. De Vries was shot on an Amsterdam city street in broad daylight on July 6, 2021, despite the fact that the Netherlands is considered one of the countries with the highest degree of press freedom in the world. The De Vries case is currently classified as Under Investigation on IPI’s Death Watch. In Greece, crime reporter Giorgos Karaivaz was shot outside his home in Athens. As of December 2021, no suspects have been publicly identified and no arrests have been made, while public information about the status of the investigation remains scarce.\nImpunity\nIPI’s analysis shows an alarmingly insufficient response from authorities to the killings, leading to high levels of impunity for crimes against journalists. In only six of the 28 cases have local police reportedly arrested suspects."
3,80d821ed-94b3-4fb6-9159-a71089c869a2,journalist risks,"Hire a Journalist, Reporter, Correspondent in Ukraine - paydesk\nGlobal Directory - All Countries - Journalists in Ukraine\nJournalists in Ukraine\n﻿\nStefan Sigaard Weichert\nHi. I am a freelance journalist living in Kyiv, Ukraine, where I report about politics, war and social stories from Eastern Europe. I 

Step 2: load all the libraries

In [32]:
from snorkel.labeling import labeling_function, PandasLFApplier
from snorkel.labeling.model.label_model import LabelModel
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import textdistance
import pandas as pd
vectorizer = TfidfVectorizer()
import re

In [33]:
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
# Download stopwords if not already present
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
nltk.download("wordnet")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Step 3: define global variables

In [34]:
# Global Variables
min_bm25 = None
max_bm25 = None
bm25 = None


Step 4: do a min-max normalisation for BM25, functions that tokenise and precompute n-grams

In [35]:
def tokenize(text):
    """Tokenizes text into lowercase words without punctuation."""
    return re.findall(r'\w+', text.lower())

def normalize_bm25(bm25_score):
    """Normalizes BM25 using min-max scaling."""
    if min_bm25 == max_bm25:
        return 0.0
    return (bm25_score - min_bm25) / (max_bm25 - min_bm25)


def tokenize_and_lemmatize(text):
    """Tokenizes text into lowercase words without punctuation and lemmatizes words."""
    tokens = re.findall(r'\w+', text.lower())  # Tokenization
    return {lemmatizer.lemmatize(word) for word in tokens} - stop_words

def get_ngrams(text, n=2):
    """Generate n-grams from text"""
    tokens = list(tokenize_and_lemmatize(text))
    return set(zip(*[tokens[i:] for i in range(n)])) if len(tokens) >= n else set()

def make_bm25_score_prob(bm25_model, doc_lookup):
    def f(x):
        query_tokens = tokenize(x.query)
        doc_idx = doc_lookup.get(x.name)  # x.name is the row index
        if doc_idx is None:
            return -1  # Abstain
        score = bm25_model.get_scores(query_tokens)[doc_idx]
        return normalize_bm25(score)
    return labeling_function()(f)

In [36]:
tokenized_docs = [tokenize(doc) for doc in input_data['unknown']]
bm25_model = BM25Okapi(tokenized_docs)
doc_lookup = dict(zip(input_data.index, range(len(input_data))))


Step 5: Create Snorkel labelling functions

In [37]:
bm25_score_prob = make_bm25_score_prob(bm25_model, doc_lookup)


@labeling_function()
def boolean_match_prob(x):
    """Returns 1 if at least one query term appears in the document."""
    return 1.0 if set(tokenize(x.query)) & set(tokenize(x.unknown)) else 0.0


@labeling_function()
def word_level_levenshtein_prob(x):
    """Computes a boosted word-level Levenshtein similarity"""
    query_words = tokenize_and_lemmatize(x.query)
    doc_words = tokenize_and_lemmatize(x.unknown)

    if not query_words or not doc_words:
        return 0.0  # Avoid division by zero

    matched_scores = []
    for q in query_words:
        similarities = [textdistance.levenshtein.normalized_similarity(q, d) for d in doc_words]
        best_match = max(similarities) if similarities else 0.0
        matched_scores.append(best_match)

    # Compute final score
    avg_similarity = sum(matched_scores) / len(matched_scores) if matched_scores else 0.0
    boosted_score = avg_similarity * (1.5 if avg_similarity > 0.8 else 1.0)

    return min(boosted_score, 1.0) #Scores in range [0,1]

@labeling_function()
def relaxed_bigram_match_prob(x):
    """Computes the proportion of query bigrams where at least one word appears in the document."""
    query_bigrams = get_ngrams(x.query, n=2)
    doc_bigrams = get_ngrams(x.unknown, n=2)

    if not query_bigrams:
        return 0.0  # Avoid division by zero

    matched_bigrams = set()
    unmatched_bigrams = set()

    for q_bigram in query_bigrams:
        # If either word in the bigram appears in the document, count it as a partial match
        if any(word in x.unknown for word in q_bigram):
            matched_bigrams.add(q_bigram)
        else:
            unmatched_bigrams.add(q_bigram)
    return len(matched_bigrams) / len(query_bigrams)  # Fraction of bigrams with at least one match


@labeling_function()
def percent_matching_query_words(x):
    """Computes the percentage of query words that appear in the document."""

    if not x.query or not x.unknown:
        return 0.0  # Handle missing data

    # Tokenization: Split words and remove punctuation
    query_words = set(re.split(r'\W+', x.query.lower())) - stop_words
    doc_words = set(re.split(r'\W+', x.unknown.lower())) - stop_words

    if not query_words:  # Avoid division by zero
        return 0.0

    # Compute match percentage
    matching_words = query_words.intersection(doc_words)
    match_percent = len(matching_words) / len(query_words)  # Fraction of matched query words

    return match_percent  # Already in [0,1] range


# Load a strong BERT model for semantic similarity
bert_model = SentenceTransformer("all-mpnet-base-v2")  # Better than MiniLM for ranking tasks

@labeling_function()
def bert_cosine_similarity(x):
    """Computes boosted BERT cosine similarity as a probability of relevance."""

    if not x.query or not x.unknown:
        return 0.0  # Handle missing data

    # Encode query and document
    query_vec = bert_model.encode(x.query, convert_to_numpy=True)
    doc_vec = bert_model.encode(x.unknown, convert_to_numpy=True)

    # Handle cases where vectors are zero (to avoid division by zero)
    if np.linalg.norm(query_vec) == 0 or np.linalg.norm(doc_vec) == 0:
        return 0.0

    # Compute cosine similarity
    cosine_sim = np.dot(query_vec, doc_vec) / (np.linalg.norm(query_vec) * np.linalg.norm(doc_vec) + 1e-9)

    # Normalize cosine similarity to [0,1]
    probability = (cosine_sim + 1) / 2

    # Apply Boosting
    if probability > 0.8:
        probability *= 1.5  # Boost high similarities
    elif probability > 0.5:
        probability *= 1.2  # Moderate boost for medium similarities

    return min(probability, 1.0)  # Keep score within [0,1]



@labeling_function()
def query_term_coverage(x):
    """Computes a relaxed query term coverage using lemmatization and synonym matching."""

    if not x.query or not x.unknown:
        return 0.0  # Handle missing data

    # Tokenization: Extract words using regex
    query_tokens = [lemmatizer.lemmatize(word) for word in re.findall(r'\b\w+\b', x.query.lower())]
    doc_tokens = [lemmatizer.lemmatize(word) for word in re.findall(r'\b\w+\b', x.unknown.lower())]

    # Create word sets
    query_words = set(query_tokens) - stop_words  # Remove stopwords
    doc_words = set(doc_tokens)

    if not query_words:  # Avoid division by zero
        return 0.0

    # === Relaxed Matching: Check for Synonyms ===
    def word_matches(word):
        if word in doc_words:
            return True  # Exact match
        synonyms = {lemma.name() for syn in wordnet.synsets(word) for lemma in syn.lemmas()}  # Get synonyms
        return bool(synonyms.intersection(doc_words))  # Check if any synonym is in the doc

    covered_terms = sum(1 for word in query_words if word_matches(word))
    coverage = covered_terms / len(query_words)  # Fraction of query terms covered by document
    if coverage > 0.7:
        coverage *= 1.8  # Strong boost
    elif coverage > 0.5:
        coverage *= 1.5  # Moderate boost
    elif coverage > 0.2:
        coverage *= 1.2  # Mild boost

    coverage = min(coverage, 1.0)  # Keep within [0,1]
    return coverage


# Global TF-IDF vectorizer with n-grams for better lexical matching
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1)

@labeling_function()
def tfidf_cosine_similarity(x):
    """Computes a boosted TF-IDF cosine similarity for document relevance probability."""

    if not x.query or not x.unknown:
        return 0.0  # Handle missing data

    # Create TF-IDF matrix
    corpus = [x.query, x.unknown]
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

    # Ensure matrix is valid
    if tfidf_matrix.shape[0] < 2 or tfidf_matrix.nnz == 0:
        return 0.0  # Avoid invalid similarity computation

    # Compute cosine similarity
    tfidf_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0, 0]

    tfidf_prob = max(tfidf_sim, 0.0)

    if tfidf_prob > 0.7:
        tfidf_prob *= 1.8  # Strong boost
    elif tfidf_prob > 0.5:
        tfidf_prob *= 1.5  # Moderate boost
    elif tfidf_prob > 0.2:
        tfidf_prob *= 1.2  # Mild boost

    return min(tfidf_prob, 1.0)  # Keep within [0,1]




6. Train Snorkel

In [38]:

lfs = [bm25_score_prob, boolean_match_prob, word_level_levenshtein_prob, tfidf_cosine_similarity, bert_cosine_similarity]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(input_data)

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=100, log_freq=10)
train_prob = label_model.predict_proba(L_train)
train_prob_relevant = train_prob[:, 1]

input_data['probability_relevant'] = train_prob_relevant

100%|██████████| 100/100 [00:00<00:00, 781.09epoch/s]


In [39]:
WMODEL = "Snorkel"

!rm -Rf run


with tracking(export_file_path='run/.metadata.yml', export_format=ExportFormat.IR_METADATA) as tracked:
    queries = set(input_data['query'].unique())

    def unknown_documents(query):
        docs = set(input_data[input_data['query'] == query]['unknown'].unique())
        return {f'{i[0]}-unkn': i[1] for i in zip(range(len(docs)), docs)}

    results = {}

    for query in tqdm(queries):
        unk = unknown_documents(query)

        # Build index
        docs = [{'docno': k, 'text': v} for k, v in unk.items()]
        indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480})
        index_ref = indexer.index(docs)
        retriever = pt.BatchRetrieve(index_ref, wmodel=WMODEL)
        rels = input_data[input_data['query'] == query][['unknown', 'probability_relevant']]
        rel_dict = dict(zip(rels['unknown'], rels['probability_relevant']))

        results[query] = rel_dict

    predictions = []
    for _, i in input_data.iterrows():
        res = results[i['query']]
        predictions.append({
            'id': i['id'],
            'probability_relevant': res.get(i['unknown'], -1)
        })
    predictions = pd.DataFrame(predictions)

predictions

100%|██████████| 13/13 [00:11<00:00,  1.17it/s]


,id,probability_relevant
0,e1786a4d-2b3d-4fcf-ad84-443acf3696e2,0.782072
1,8e64b35c-60be-4292-9542-c4639a749336,0.000311
2,a8ff3f18-9b80-40b9-b529-799ed3367980,0.318625
3,80d821ed-94b3-4fb6-9159-a71089c869a2,0.318625
4,709154d7-13c6-492c-b977-07396510909c,0.318625
...,...,...
1013,527a986a-c2d9-46b6-8a14-3f305beaf7fd,0.782072
1014,4c8dee20-f7cb-44b3-ae69-8ea00a7e2ede,0.782072
1015,4a773373-3f0b-4821-a78e-86ce9bc429da,0.782072
1016,369ce954-c156-485c-8cdc-37e0137287d2,0.782072


In [40]:
wows_evaluate(
    predictions,
    DATASET_ID,
    tracking_results=tracked,
    upload=True,
    system_name="Snorkel",
    system_description=f'We use the PyTerrier retrieval model {WMODEL} to assign the probability that an unknown document is relevant. We rank all unknown documents that are to-be judged against the query. The probability that a document is relevant is then the min-max normalized rank of an unknown document in the ranking of all unknown documents to the query.'
)


Run uploaded to TIRA. Claim ownership via: https://www.tira.io/claim-submission/3ecb6039-9455-4636-86a5-8645c806559d
No truth data is available yet. The evaluation is possible after the deadline when the truth data was published.
